In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from bokeh.io import show, output_notebook
from bokeh.models import (ColumnDataSource,
                            HoverTool)
from bokeh.plotting import figure
output_notebook()

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


Loading BokehJS ...

Data:

In [10]:
## GIS from Ron
lccTrees = gpd.read_file("filesMap/LCCtrees.geojson")
lccSensors = gpd.read_file("filesMap/LCCSensors.geojson")

## Herrmann et al. sampled these trees before:
prevTrees = [ "129.0", "317.0", "733.0", "33.0" , "513.1", "754.0",
                "444.0", "455.0", "517.1" ]
##  clear up the LI/WLI difference
lccSensors['BAUMART'] = lccSensors.BAUMART.str.replace('LI','WLI')
## make the unique IDs compatible:
lccTrees['Tree_ID'] = lccTrees['Tree_ID'].str.replace('K','')
## make BHD into numeric column
lccTrees['BHD_2020'] = lccTrees['BHD_2020'].replace('na', np.NaN)
lccTrees['BHD_2020'] = lccTrees['BHD_2020'].str.replace(',','.')
lccTrees['BHD_2020'].dropna(axis='index', how='any', inplace=True)
lccTrees['BHD_2020'] = lccTrees['BHD_2020'].astype('float64')
## let's make a Series for markersizes from BHD (they are too big
## as is)
lccTrees['BHDmarkersize'] = lccTrees['BHD_2020'].apply(np.log10)
## we need a separate data source for sensors, I think:
## ring size for sensors:
lccTrees['sensorSize'] = lccTrees['BHDmarkersize'].apply(lambda a: a*1.4)
## sensor df for CDS
sensfilt = lccTrees.Tree_ID.isin(lccSensors.Name)
sensDF = lccTrees.loc[sensfilt,:]

## future trees are most likely trees that:
## aren't maple, are previously-sampled, and have sensors.
aa = lccTrees[~lccTrees['spec'].isin(['BAH'])].copy()
bb = aa[aa['Tree_ID'].isin(prevTrees)].copy()
cc = bb[bb.Tree_ID.isin(lccSensors.Name)].copy()
cc.drop('geometry',axis='columns', inplace=True)
mostLikelySampleAgainDF = cc

## or they are ash trees with sensors on them:
aa = lccTrees[lccTrees['spec'].isin(['GES'])].copy()
bb = aa[aa.Tree_ID.isin(lccSensors.Name)].copy()
bb.drop('geometry',axis='columns', inplace=True)
mostLikelyAshesDF = bb

## make our CDSes
sensorCDS = ColumnDataSource(data=sensDF.drop('geometry', axis='columns'))
mostLikelySampleAgainCDS = ColumnDataSource(data=mostLikelySampleAgainDF)
mostLikelyAshesCDS = ColumnDataSource(data=mostLikelyAshesDF)
lccCDS = ColumnDataSource(data=lccTrees.drop('geometry', axis='columns'))

In [11]:
p = figure (title='LCC trees ',
           plot_height=800,
            aspect_ratio=1,
            toolbar_location='below',
            tools='pan, wheel_zoom, box_zoom, reset'
           )
## laydown sensor circles
sensorCircs = p.circle('E_UTM', 'N_UTM', source=sensorCDS,
                    fill_alpha=0,
                    line_color='#0000FF',
                    line_width=3,
                    line_alpha=1,
                    radius='sensorSize',
                    legend_label='sensor',
                    )
## background tree circles
trCircs = p.circle('E_UTM', 'N_UTM', source=lccCDS,
                    fill_color='#a7a7a7',
                    radius='BHDmarkersize',
                    alpha=0.3,
                    )
## non-maple, previously-sampled, probably soon-to-be sampled trees:
mlsaCircs = p.circle('E_UTM', 'N_UTM', source=mostLikelySampleAgainCDS,
                    fill_color='red',
                    radius='BHDmarkersize',
                    alpha=1,
                    legend_label='probable resample'
                    )
## ashes with sensors:
ashCircs = p.circle('E_UTM', 'N_UTM', source=mostLikelyAshesCDS,
                    fill_color='yellow',
                    radius='BHDmarkersize',
                    alpha=1,
                    legend_label='ash trees with sensor'
                    )


p.add_tools(HoverTool(renderers = [trCircs],
                      tooltips = [('K','@Tree_ID'),
                                  ('spec','@spec'),]))
show(p)
